This notebook querries the gaia sources around the known GEMS and computes the 3D separation from the target. I found that about 1 in 4 among the known GEMS have a nearby companion. 

TODO: 
* check which GEMS have companions not reported in literature (discovery paper)
* check if these are comoving companions
* check if theory supports that giant planets are more likely to form around binary star disk

In [1]:
import sys
sys.path.insert(0, '..')

In [2]:
from toi5671 import Star

In [3]:
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive
import pandas as pd
from pathlib import Path

redownload = False

fp = '../data/nexsci_data.csv'
if redownload or not Path(fp).exists():
    #pstable combines data from the Confirmed Planets and Extended Planet Parameters tables
    tab = NasaExoplanetArchive.query_criteria(table="pscomppars", 
    #                                          where="discoverymethod like 'Transit'"
    )
    df = tab.to_pandas()                                        
    df.to_csv(fp, index=True)
    
else:
    df = pd.read_csv(fp, index_col=0)
    df.index.name = 'index'
df.head()

/var/folders/4s/07r41mpn76n_0fykwqyjgc840000gn/T/ipykernel_94374/1112017024.py:17: DtypeWarning: Columns (70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fp, index_col=0)


,pl_name,pl_letter,hostname,hd_name,hip_name,tic_id,disc_pubdate,disc_year,discoverymethod,disc_locale,disc_facility,disc_instrument,disc_telescope,disc_refname,ra,rastr,dec,decstr,glon,glat,elon,elat,ra_reflink,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbperstr,pl_orbper_reflink,pl_orblpererr1,pl_orblper,pl_orblpererr2,pl_orblperlim,pl_orblperstr,pl_orblper_reflink,pl_orbsmax,pl_orbsmaxerr1,pl_orbsmaxerr2,pl_orbsmaxlim,pl_orbsmaxstr,...,st_rad,st_raderr1,st_raderr2,st_radlim,st_radstr,st_rad_reflink,ttv_flag,ptv_flag,tran_flag,rv_flag,ast_flag,obm_flag,micro_flag,etv_flag,ima_flag,pul_flag,sy_snum,sy_pnum,sy_mnum,st_nphot,st_nrvc,st_nspec,pl_nespec,pl_ntranspec,pl_nnotes,sy_pm,sy_pmerr1,sy_pmerr2,sy_pmstr,sy_pm_reflink,sy_pmra,sy_pmraerr1,sy_pmraerr2,sy_pmrastr,x,y,z,htm20,sky_coord.ra,sky_coord.dec
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,OGLE-2016-BLG-1227L b,b,OGLE-2016-BLG-1227L,NaN,NaN,NaN,2020-03,2020,Microlensing,Ground,OGLE,OGLE CCD Array,1.3 m Warsaw University Telescope,<a refstr=HAN_ET_AL__2020 href=https://ui.adsa...,265.597125,17h42m23.31s,-33.759778,-33d45m35.2s,355.52875,-1.93858,266.27974,-10.37734,<a refstr=HAN_ET_AL__2020 href=https://ui.adsa...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,3.4000,2.1000,-1.0000,0,"<div><span class=""supersubNumber"">3.4</span> <...",...,NaN,NaN,NaN,0,NaN,NaN,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,4,NaN,NaN,NaN,NaN,<a refstr=HAN_ET_AL__2020 href=https://ui.adsa...,NaN,NaN,NaN,NaN,-0.063824,-0.828921,-0.555712,-768415656,265.597125,-33.759778
1,Kepler-276 c,c,Kepler-276,NaN,NaN,TIC 138213510,2014-02,2013,Transit,Space,Kepler,Kepler CCD Array,0.95 m Kepler Telescope,<a refstr=XIE_2014 href=https://ui.adsabs.harv...,293.568197,19h34m16.37s,39.036312,+39d02m10.72s,72.30607,9.13322,307.64188,59.43368,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,31.884000,NaN,NaN,0,31.884,<a refstr=XIE_2014 href=https://ui.adsabs.harv...,NaN,NaN,NaN,0,NaN,NaN,0.1994,NaN,NaN,0,.1994,...,1.03,0.45,-0.10,0,"<div><span class=""supersubNumber"">1.03</span> ...",<a refstr=XIE_2014 href=https://ui.adsabs.harv...,1,0,1,0,0,0,0,0,0,0,1,3,0,0,0,0,0,0,1,8.070260,0.051510,-0.051510,8.0702604&plusmn;0.0515101,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,7.99553,0.051581,-0.051581,7.9955300&plusmn;0.0515805,0.310575,-0.711954,0.629813,1302657699,293.568197,39.036312
2,Kepler-829 b,b,Kepler-829,NaN,NaN,TIC 123451768,2016-05,2016,Transit,Space,Kepler,Kepler CCD Array,0.95 m Kepler Telescope,<a refstr=MORTON_ET_AL__2016 href=https://ui.a...,282.332831,18h49m19.88s,42.463813,+42d27m49.73s,71.95093,18.29325,291.86447,64.96975,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,6.883376,0.000020,-0.000020,0,6.88337562&plusmn1.963e-05,<a refstr=MORTON_ET_AL__2016 href=https://ui.a...,NaN,NaN,NaN,0,NaN,NaN,0.0678,NaN,NaN,0,.0678,...,0.97,0.20,-0.08,0,"<div><span class=""supersubNumber"">0.9700</span...",<a refstr=MORTON_ET_AL__2016 href=https://ui.a...,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,8.308909,0.041298,-0.041298,8.3089089&plusmn;0.0412978,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,7.93777,0.040833,-0.040833,7.9377700&plusmn;0.0408326,0.157566,-0.720680,0.675124,-792260935,282.332831,42.463813
3,K2-283 b,b,K2-283,NaN,NaN,TIC 266017624,2018-12,2018,Transit,Space,K2,Kepler CCD Array,0.95 m Kepler Telescope,<a refstr=LIVINGSTON_ET_AL__2018 href=https://...,13.194368,00h52m46.65s,9.692918,+09d41m34.51s,123.48272,-53.17740,15.90137,3.72545,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,1.921036,0.000051,-0.000053,0,"<div><span class=""supersubNumber"">1.921036</sp...",<a refstr=LIVINGSTON_ET_AL__2018 href=https://...,NaN,NaN,NaN,0,NaN,NaN,0.0291,0.0003,-0.0003,0,0.0291&plusmn0.0003,...,0.82,0.01,-0.01,0,0.82&plusmn0.01,<a refstr=LIVINGSTON_ET_AL__2018 href=https://...,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,11.110176,0.053185,-0.053185,11.1101761&plusmn;0.0531852,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,-10.97340,0.053624,-0.053624,-10.9734

In [9]:
#GEMS
idx = (df.st_teff<=4000) & (df.pl_radj > 0.4) & (df.pl_ratdor < 20) & (df.discoverymethod=="Transit")
gems = df[idx]
gems.shape

(26, 375)

In [13]:
gaia_sources.columns

Index(['solution_id', 'designation', 'source_id', 'random_index', 'ref_epoch',
       'ra', 'ra_error', 'dec', 'dec_error', 'parallax', 'parallax_error',
       'parallax_over_error', 'pmra', 'pmra_error', 'pmdec', 'pmdec_error',
       'ra_dec_corr', 'ra_parallax_corr', 'ra_pmra_corr', 'ra_pmdec_corr',
       'dec_parallax_corr', 'dec_pmra_corr', 'dec_pmdec_corr',
       'parallax_pmra_corr', 'parallax_pmdec_corr', 'pmra_pmdec_corr',
       'astrometric_n_obs_al', 'astrometric_n_obs_ac',
       'astrometric_n_good_obs_al', 'astrometric_n_bad_obs_al',
       'astrometric_gof_al', 'astrometric_chi2_al', 'astrometric_excess_noise',
       'astrometric_excess_noise_sig', 'astrometric_params_solved',
       'astrometric_primary_flag', 'astrometric_weight_al',
       'astrometric_pseudo_colour', 'astrometric_pseudo_colour_error',
       'mean_varpi_factor_al', 'astrometric_matched_observations',
       'visibility_periods_used', 'astrometric_sigma5d_max',
       'frame_rotator_object_type',

In [ ]:
from tqdm import tqdm
from astropy.coordinates import SkyCoord
from astropy.coordinates import Distance
import astropy.units as u

gaia = {}
cols = ["ra","dec","parallax","pmra","pmdec","radial_velocity","phot_g_mean_mag","bp_rp","distance"]
for name in tqdm(gems.hostname):
    print(name)
    try:
        star = Star(name)
        gaia_sources = star.get_gaia_sources(rad_arcsec=60)
        # print(gaia_sources[cols])
        
        s = gaia_sources[cols].dropna(subset=["parallax"])
        s = s[s["parallax"]>0]
        coords = SkyCoord(ra=s.ra.values*u.deg, 
                         dec=s.dec.values*u.deg,
                         pm_ra_cosdec=s.pmra.values*u.mas/u.yr,
                         pm_dec=s.pmdec.values*u.mas/u.yr,
                         radial_velocity=s.radial_velocity.values*u.km/u.second,
                         distance=Distance(parallax=s.parallax.values*u.mas)
                        )
        s['distance_3d'] = coords.separation_3d(coords[0]).value
        gaia[name] = s
    except Exception as e:
        print(e)

In [86]:
ls -lh ../data/gems_companions.h5

-rw-r--r--@ 1 jpdl  staff   125K Apr 22 19:07 ../data/gems_companions.h5


In [ ]:
import flammkuchen as fk

fk.save("../data/gems_companions.h5", gaia)

In [80]:
data = fk.load("../data/gems_companions.h5")
has_nearby_companion = []
for name in data:
    print(name)
    print(data[name].sort_values(by="distance_3d").head(3))
    if len(data[name])>1 and data[name].sort_values(by="distance_3d").iloc[1].distance_3d<100:
        has_nearby_companion.append(name)

HATS-6
          ra        dec  parallax  ...     bp_rp   distance  distance_3d
0  88.146808 -19.031627  5.877420  ...  2.025865   0.000017     0.000000
1  88.150045 -19.032647  2.226236  ...  1.930374  11.612620   279.045910
9  88.133224 -19.033950  1.916835  ...  0.791909  46.977525   351.550552

[3 rows x 10 columns]
HATS-71
          ra        dec  parallax  ...     bp_rp  distance  distance_3d
0  15.551889 -61.756133  7.102862  ...  2.556134  0.000009          0.0

[1 rows x 10 columns]
HATS-74 A
           ra        dec  parallax  ...     bp_rp   distance  distance_3d
0  171.014792 -19.556959  3.313652  ...  2.431289   0.000022     0.000000
8  171.003481 -19.544653  3.330078  ...  2.896433  58.609800     1.491008
1  171.014973 -19.556800  3.802305  ...       NaN   0.840136    38.783422

[3 rows x 10 columns]
HATS-75
          ra        dec  parallax  ...     bp_rp   distance  distance_3d
0  60.949392 -25.408921  5.100383  ...  2.165428   0.000006     0.000000
1  60.966633 -25.414

In [81]:
has_nearby_companion

['HATS-74 A', 'Kepler-1624', 'TOI-3235', 'TOI-3714', 'TOI-3757', 'TOI-3984 A']

In [83]:
len(has_nearby_companion)/len(gems)

0.23076923076923078